<a href="https://colab.research.google.com/github/aanskaitis/CSC3831/blob/main/DataLinkage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fuzzymatcher

     |████████████████████████████████| 50 kB 4.2 MB/s 
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13918 sha256=bf6cebb36c9ab8a54cb3b86f6b0d8943b5ed903617353b31397737cb6e3e55ca
  Stored in directory: /root/.cache/pip/wheels/1d/a8/cb/6f8902aa5457bd71344e00665c230e9c45255b3f57f2194a0f
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149856 sha256=d220b13086b6ee15a65d1fb7193d049cd9cdd69b830691bfb6bdc162d0199aa2
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built metaphone python-Levenshtein


In [ ]:
pip install recordlinkage

     |████████████████████████████████| 944 kB 9.6 MB/s 
     |████████████████████████████████| 132 kB 55.2 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73973 sha256=1e35942ce7cabed8f02f45766793b5a827b86bca2f72ef2dedd4c9404f2f075f
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [ ]:
import pandas as pd
import fuzzymatcher
import recordlinkage
import time

In [ ]:
df_ACM = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv', index_col=0)
df_DBLP = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', index_col=0)

In [ ]:
df_ACM.head()

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999


In [ ]:
df_DBLP.head()

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995


Check how many missing values do we have.

In [ ]:
df_ACM.isna().sum()

id          0
title       0
authors    14
venue       0
year        0
dtype: int64

In [ ]:
df_DBLP.isna().sum()

id         0
title      0
authors    0
venue      0
year       0
dtype: int64

We have 14 missing authors in ACM dataset. Check rows with missing values to see if we can draw any conclusions about their missingness.

In [ ]:
df_ACM[df_ACM['authors'].isna()]

,id,title,authors,venue,year
884,671838,High-Performance and Scalability through Appli...,NaN,Very Large Data Bases,2000
1146,673478,MineSet(tm): A System for High-End Data Mining...,NaN,Very Large Data Bases,1996
2128,959075,Database principles,NaN,ACM SIGMOD Record,2003
2129,959077,Book reviews,NaN,ACM SIGMOD Record,2003
2130,959082,Reports,NaN,ACM SIGMOD Record,2003
2131,945726,Special topic section on peer to peer data man...,NaN,ACM SIGMOD Record,2003
2132,945738,Standards,NaN,ACM SIGMOD Record,2003
2133,945740,Book reviews,NaN,ACM SIGMOD Record,2003
2134,959063,Special section on sensor network technology a...,NaN,ACM SIGMOD Record,2003
2136,758376,Author Index,NaN,Very Large Data Bases,2000


Some titles like 'Author Index' or 'Standards' might suggest that these are not actual publications and were accidentally included in the dataset, although we cannot guarantee that. Remove rows with missing authors so that it doesn't interfere with our record linkage process. 

In [ ]:
df_ACM = df_ACM.dropna()

Check how many records we have left in each dataframe.

In [ ]:
print(f'Number of records in ACM dataset: {len(df_ACM)}')
print(f'Number of records in DBLP dataset: {len(df_DBLP)}')

Number of records in ACM dataset: 2280
Number of records in DBLP dataset: 2616


Print several authors from both dataframes starting with AL to get a general view

In [ ]:
mask = df_ACM['authors'].str.startswith('Al')
df_ACM[mask].sort_values(by='authors')[['authors', 'title']].head(20)

,authors,title
1433,"Alain Pirotte, Esteban Zim&#225;nyi, David Mas...",Materialization: A Powerful and Ubiquitous Abs...
2222,"Alan Demers, Johannes Gehrke, Rajmohan Rajaram...",The Cougar Project: a work-in-progress report
1885,"Albert D'Andrea, Phil Janus",UniSQL's next-generation object-relational dat...
654,"Albertas Caplinskas, Johann Eder, Olegas Vasil...",Advances in databases and information systems:...
1478,"Alberto Belussi, Christos Faloutsos",Estimating the Selectivity of Spatial Queries ...
737,"Alberto H. F. Laender, Berthier A. Ribeiro-Net...",A brief survey of web data extraction tools
1384,"Alberto O. Mendelzon, Alejandro A. Vaisman",Temporal Queries in OLAP
2121,"Alberto d'Onofrio, Elaheh Pourabbas",Modelling temporal thematic map contents
568,"Albrecht Schmidt, Florian Waas, Martin Kersten...",Why and how to benchmark XML databases
2237,"Alejandro Guti&#233;rrez, Philippe Pucheral, H...",Database Graph Views: A Practical Model to Man...


In [ ]:
mask = df_DBLP['authors'].str.startswith('Al')
df_DBLP[mask].sort_values(by='authors')[['authors', 'title']].head(20)

,authors,title
53,"Alain Pirotte, Esteban Zimányi, Tatiana Yakush...",Materialization: A Powerful and Ubiquitous Abs...
57,"Alan F. Smeaton, Mark Roantree",Research in Information Managment at Dublin Ci...
2571,"Alan Jay Smith, Vigyan Singhal",Analysis of Locking Behavior in Three Real Dat...
2075,"Alan Sussman, Anurag Acharya, Joel H. Saltz, C...",T2: A Customizable Parallel Database for Multi...
1832,"Albertas Caplinskas, Johann Eder, Olegas Vasil...",Advances in Databases and Information Systems ...
123,"Alberto Lerner, Dennis Shasha","AQuery: Query Language for Ordered Data, Optim..."
2111,Alberto O. Mendelzon,Information Director's Message
285,Alberto O. Mendelzon,Information Director's Message
2361,"Alberto O. Mendelzon, Alejandro A. Vaisman",Temporal Queries in OLAP
114,"Alberto O. Mendelzon, Davood Rafiei",Similarity-Based Queries for Time Series Data


There are some matching authors, although ACM dataset has non standard English language letters replaced with character codes while DBLP dataset authors contain those non standard letters without character codes. Try simple left join and check the results.

In [ ]:
df_merged = df_ACM.merge(df_DBLP, on='authors', how='left', suffixes=('_ACM', '_DBLP'))

In [ ]:
len(df_merged)

2987

In [ ]:
df_merged[['authors', 'title_ACM', 'title_DBLP']].head(20)

,authors,title_ACM,title_DBLP
0,"Gottfried Vossen, Mathias Weske",The WASA2 object-oriented workflow management ...,NaN
1,"Isabel F. Cruz, Kimberly M. James",A user-centered interface for querying distrib...,NaN
2,"Athman Bouguettaya, Boualem Benatallah, Lily H...","World Wide Database-integrating the Web, CORBA...",NaN
3,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",XML-based information mediation with MIX,NaN
4,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",The CCUBE constraint object-oriented database ...,NaN
5,"Phillippe Bonnet, Kyle Buza, Zhiyuan Chan, Vic...",The Cornell Jaguar project: adding mobility to...,NaN
6,"Nick Roussopoulos, Yannis Kotidis, Yannis Sism...",The active MultiSync controller of the cubetre...,NaN
7,"Michael B&#246;hlen, Linas Bukauskas, Curtis D...",The Jungle database search engine,NaN
8,"N. R. Jennings, T. J. Norman, P. Faratin",ADEPT: an agent-based approach to business pro...,NaN
9,"Benny Reich, Israel Ben-Shaul",A componentized architecture for dynamic elect...,NaN


In [ ]:
df_merged[df_merged['id_DBLP'].notnull()].sort_values(by='authors').head(25)

,id_ACM,title_ACM,authors,venue_ACM,year_ACM,id_DBLP,title_DBLP,venue_DBLP,year_DBLP
2795,756664,Large Databases for Remote Sensing and GIS,A. R. Dasgupta,Very Large Data Bases,1996,conf/vldb/Dasgupta96,Large Databases for Remote Sensing and GIS,VLDB,1996.0
1860,671869,Rainbow: Distributed Database System for Class...,"Abdelsalam Helal, Hua Li",Very Large Data Bases,2000,conf/vldb/HelalL00,Rainbow: Distributed Database System for Class...,VLDB,2000.0
1641,765532,Integrated document caching and prefetching in...,"Achim Kraiss, Gerhard Weikum",The VLDB Journal &mdash; The International Jou...,1998,journals/vldb/KraissW98,Integrated Document Caching and Prefetching in...,VLDB J.,1998.0
1522,673664,Vertical Data Migration in Large Near-Line Doc...,"Achim Kraiss, Gerhard Weikum",Very Large Data Bases,1997,conf/vldb/KraissW97,Vertical Data Migration in Large Near-Line Doc...,VLDB,1997.0
1642,765532,Integrated document caching and prefetching in...,"Achim Kraiss, Gerhard Weikum",The VLDB Journal &mdash; The International Jou...,1998,conf/vldb/KraissW97,Vertical Data Migration in Large Near-Line Doc...,VLDB,1997.0
1521,673664,Vertical Data Migration in Large Near-Line Doc...,"Achim Kraiss, Gerhard Weikum",Very Large Data Bases,1997,journals/vldb/KraissW98,Integrated Document Caching and Prefetching in...,VLDB J.,1998.0
2784,671348,Microsoft English Query 7.5: Automatic Extract...,Adam Blum,Very Large Data Bases,1999,conf/vldb/Blum99,Microsoft English Query 7.5: Automatic Extract...,VLDB,1999.0
2089,202668,An annotated bibliography of benchmarks for ob...,Akmal B. Chaudhri,ACM SIGMOD Record,1995,journals/sigmod/Chaudhri95,An Annotated Bibliography of Benchmarks for Ob...,SIGMOD Record,1995.0
2088,202668,An annotated bibliography of benchmarks for ob...,Akmal B. Chaudhri,ACM SIGMOD Record,1995,journals/sigmod/Chaudhri98,Workshop Report on Experiences Using Object Da...,SIGMOD Record,1998.0
2354,273247,Workshop report on experiences using object da...,Akmal B. Chaudhri,ACM SIGMOD Record,1998,journals/sigmod/Chaudhri95,An Annotated Bibliography of Benchmarks for Ob...,SIGMOD Record,1995.0


Some were matched, because the authors were the same, although the publications were different. Like this one:

In [ ]:
df_merged.iloc[[1672, 2921]][['authors', 'title_ACM', 'year_ACM', 'title_DBLP', 'year_DBLP']]

,authors,title_ACM,year_ACM,title_DBLP,year_DBLP
1672,"Alexandra Poulovassilis, Carol Small",Algebraic query optimisation for database prog...,1996,Investigation of Algebraic Query Optimisation ...,1994.0
2921,"Alexandra Poulovassilis, Carol Small",Investigation of Algebraic Query Optimisation ...,1994,Investigation of Algebraic Query Optimisation ...,1994.0


Two different publications from the same authors from the ACM dataset matched with the same one publication from the DBLP dataset.

Check how many have we matched.

In [ ]:
len(df_merged[df_merged['id_DBLP'].notnull()])

1419

We still have some work to do:
*   There might be spelling mistakes preventing us from matching authors.
*   Publications made by multiple authors could have order of authors switched compared to the other dataset.
*   Some already matched ones are wrong, because of the reason mentioned above.

Try fuzzymatcher and see if we get better results.

## Fuzzy Matcher

In [ ]:
# Both datasets have the same column names, 
# but for clarity purposes we'll write them out for both

# Columns to match from left dataset
left_on = ['title', 'authors', 'venue', 'year']

# Columns to match from right dataset
right_on = ['title', 'authors', 'venue', 'year']

In [ ]:
# link table
df_linked = fuzzymatcher.link_table(df_ACM, df_DBLP, left_on, right_on, 
                                    left_id_col='id', right_id_col='id')

In [ ]:
df_linked.head()

,__id_left,__id_right,match_score,match_rank,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
0,304586,conf/sigmod/VossenW99,2.559548,1,The WASA2 object-oriented workflow management ...,The WASA2 Object-Oriented Workflow Management ...,"Gottfried Vossen, Mathias Weske","Mathias Weske, Gottfried Vossen",International Conference on Management of Data,SIGMOD Conference,1999,1999
1,304587,conf/sigmod/CruzJ99,2.721471,1,A user-centered interface for querying distrib...,A User-Centered Interface for Querying Distrib...,"Isabel F. Cruz, Kimberly M. James","Kimberly M. James, Isabel F. Cruz",International Conference on Management of Data,SIGMOD Conference,1999,1999
2,304589,conf/sigmod/BouguettayaBH99,4.568259,1,"World Wide Database-integrating the Web, CORBA...","World Wide Database - Integrating the Web, COR...","Athman Bouguettaya, Boualem Benatallah, Lily H...","Kevin Smith, Athman Bouguettaya, Mourad Ouzzan...",International Conference on Management of Data,SIGMOD Conference,1999,1999
3,304590,conf/sigmod/BaruGLMPVC99,3.630595,1,XML-based information mediation with MIX,XML-Based Information Mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...","Yannis Papakonstantinou, Richard Marciano, Cha...",International Conference on Management of Data,SIGMOD Conference,1999,1999
4,304582,conf/sigmod/BrodskySCE99,2.969856,1,The CCUBE constraint object-oriented database ...,The CCUBE Constraint Object-Oriented Database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...","Alexander Brodsky, Pavel A. Exarkhopoulo, Vict...",International Conference on Management of Data,SIGMOD Conference,1999,1999


Looks good, we already see some matches here.

In [ ]:
mask = df_linked['authors_left'].str.contains('Alexander S.')
df_linked[mask].sort_values(by='match_score', ascending=False)

,__id_left,__id_right,match_score,match_rank,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
18328,564758,conf/sigmod/SzalayGTKMRSv02,6.679060,1,The SDSS skyserver: public access to the sloan...,The SDSS skyserver: public access to the sloan...,"Alexander S. Szalay, Jim Gray, Ani R. Thakar, ...","Tanu Malik, Jordan Raddick, Alexander S. Szala...",International Conference on Management of Data,SIGMOD Conference,2002,2002
5879,335439,conf/sigmod/SzalayGKT00,5.072419,1,Designing and mining multi-terabyte astronomy ...,Designing and Mining Multi-Terabyte Astronomy ...,"Alexander S. Szalay, Peter Z. Kunszt, Ani Thak...","Alexander S. Szalay, Donald R. Slutz, Ani Thak...",International Conference on Management of Data,SIGMOD Conference,2000,2000
5880,335439,conf/sigmod/Carey00,-3.018442,2,Designing and mining multi-terabyte astronomy ...,On XML and Databases: Where's the Beef? (Panel...,"Alexander S. Szalay, Peter Z. Kunszt, Ani Thak...",Michael J. Carey,International Conference on Management of Data,SIGMOD Conference,2000,2000
5881,335439,conf/sigmod/Gal00,-3.599059,3,Designing and mining multi-terabyte astronomy ...,"Data Management in eCommerce: The Good, the Ba...","Alexander S. Szalay, Peter Z. Kunszt, Ani Thak...",Avigdor Gal,International Conference on Management of Data,SIGMOD Conference,2000,2000
5882,335439,conf/sigmod/GoldmanW00,-3.634546,4,Designing and mining multi-terabyte astronomy ...,WSQ/DSQ: A Practical Approach for Combined Que...,"Alexander S. Szalay, Peter Z. Kunszt, Ani Thak...","Roy Goldman, Jennifer Widom",International Conference on Management of Data,SIGMOD Conference,2000,2000


Let's perform the matching:

In [ ]:
df_matched = fuzzymatcher.fuzzy_left_join(df_ACM, df_DBLP, left_on, right_on,
                                          left_id_col='id', right_id_col='id')

36.597973346710205

Reorder columns for clarity:

In [ ]:
df_matched.columns

Index(['best_match_score', '__id_left', '__id_right', 'id_left', 'title_left',
       'authors_left', 'venue_left', 'year_left', 'id_right', 'title_right',
       'authors_right', 'venue_right', 'year_right'],
      dtype='object')

In [ ]:
cols = ['best_match_score', 'authors_left', 'authors_right',
        'title_left', 'title_right', 'year_left',
        'year_right', 'venue_left', 'venue_right']

Let's look at the highest score matches:

In [ ]:
df_matched[cols].sort_values(by='best_match_score', ascending=False).head()

,best_match_score,authors_left,authors_right,title_left,title_right,year_left,year_right,venue_left,venue_right
25237,9.284321,"Michael J. Carey, Steve Kirsch, Mary Roth, Ber...","Runping Qi, Kuan Yee, Brian Sterling, Nicolas ...",The Propel Distributed Services Platform,The Propel Distributed Services Platform,2001,2001,Very Large Data Bases,VLDB
34559,9.233475,"Eugene Inseok Chong, Jagannathan Srinivasan, S...","Mahesh Jagannath, Eugene Inseok Chong, Aravind...",A mapping mechanism to support bitmap index an...,A mapping mechanism to support bitmap index an...,2003,2003,ACM SIGMOD Record,SIGMOD Record
29283,9.165640,"Richard Thomas Snodgrass, Ilsoo Ahn, Gadi Aria...","Wolfgang Käfer, Curtis E. Dyreson, Arie Segev,...",TSQL2 language specification,TSQL2 Language Specification,1994,1994,ACM SIGMOD Record,SIGMOD Record
28619,9.091846,"Richard Thomas Snodgrass, Ilsoo Ahn, Gad Ariav...","James Clifford, Ramez Elmasri, Michael D. Soo,...",A TSQL2 tutorial,A TSQL2 Tutorial,1994,1994,ACM SIGMOD Record,SIGMOD Record
427,8.605697,"Wen-Syan Li, Quoc Vu, Edward Chang, Divyakant ...","Kazuyuki Shimazu, Sougata Mukherjea, Yi-Leh Wu...",PowerBookmarks: a system for personalizable We...,PowerBookmarks: A System for Personalizable We...,1999,1999,International Conference on Management of Data,SIGMOD Conference


Now, at the worst ones:

In [ ]:
df_matched[cols].sort_values(by='best_match_score').head()

,best_match_score,authors_left,authors_right,title_left,title_right,year_left,year_right,venue_left,venue_right
19602,-1.972738,Marianne Winslett,Marianne Winslett,David DeWitt speaks out: on rethinking the CS ...,Gio Wiederhold Speaks Out on Moving into Acade...,2002,2001,ACM SIGMOD Record,SIGMOD Record
29397,-1.682680,"Curtis Dyreson, Fabio Grandi, Wolfgang K&#228;...","James Clifford, Ramez Elmasri, Michael D. Soo,...",A consensus glossary of temporal database conc...,A TSQL2 Tutorial,1994,1994,ACM SIGMOD Record,SIGMOD Record
5656,-1.278196,Val Huber,Ronald G. Ross,Challenges in automating declarative business ...,Expressing Business Rules,2000,2000,International Conference on Management of Data,SIGMOD Conference
34030,-1.271728,Zachary Ives,Objectivity,Abstracts of invited industrial track presenta...,Objectivity Industrial Exhibit,2003,1998,International Conference on Management of Data,VLDB
5447,-1.263943,Eric Kintzer,Eric A. Brewer,Going beyond personalization: rule engines at ...,Delivering High Availability for Inktomi Searc...,2000,1998,International Conference on Management of Data,SIGMOD Conference


Now, at ones somewhere in the middle:

In [ ]:
mask = (df_matched['best_match_score'] > 0.7) & (df_matched['best_match_score'] < 0.9)
df_matched[cols][mask]

,best_match_score,authors_left,authors_right,title_left,title_right,year_left,year_right,venue_left,venue_right
537,0.880783,Dan Suciu,Dan Suciu,Managing Web data,Managing Web Data,1999,1999,International Conference on Management of Data,SIGMOD Conference
670,0.774901,Anand Rajaraman,Anand Rajaraman,E-commerce database issues and experience: (ta...,E-Commerce Database Issues and Experience,1999,1999,International Conference on Management of Data,SIGMOD Conference
683,0.776535,CORPORATE TimesTen Team,Times-Ten Team,In-memory data management for consumer transac...,In-Memory Data Management for Consumer Transac...,1999,1999,International Conference on Management of Data,SIGMOD Conference
2298,0.824726,Larry J. Ellison,Larry Ellison,Keynote address,Keynote Address,1995,1995,International Conference on Management of Data,SIGMOD Conference
3107,0.899367,Pillip A. Bernstein,Philip A. Bernstein,Repository system engineering,Repository System Engineering,1996,1996,International Conference on Management of Data,SIGMOD Conference
3108,0.823825,Daniel A. Keim,Daniel A. Keim,databases and visualization,Databases and Visualization,1996,1996,International Conference on Management of Data,SIGMOD Conference
3114,0.713977,Jiawei Han,Jiawei Han,Data mining techniques,Data Mining Techniques,1996,1996,International Conference on Management of Data,SIGMOD Conference
3784,0.776535,Asuman Dogac,Asuman Dogac,Data management issues in electronic commerce:...,Guest Editor's Introduction,2002,2002,ACM SIGMOD Record,SIGMOD Record
4395,0.849605,"C. Mohan, Larry Cable, Matthieu Devin, Scott D...",C. Mohan,Application servers (panel session): born-agai...,Application Servers: Born-Again TP Monitors fo...,2001,2001,International Conference on Management of Data,SIGMOD Conference
5902,0.818986,"B. P&#244;ssas, M. Carvalho, R. Resende, W. Me...","Márcio de Carvalho, Rodolfo F. Resende, Bruno ...",Using quantitative information for efficient a...,Using Quantitative Information for Efficient A...,2000,2000,ACM SIGMOD Record,SIGMOD Record


In [ ]:
mask = (df_matched['best_match_score'] > 0.2) & (df_matched['best_match_score'] < 0.4)
df_matched[cols][mask]

,best_match_score,authors_left,authors_right,title_left,title_right,year_left,year_right,venue_left,venue_right
556,0.359636,Dirk D&#252;llmann,Dirk Düllmann,Petabyte databases,Petabyte Databases,1999,1999,International Conference on Management of Data,SIGMOD Conference
569,0.309773,Bassel Ojjeh,Bassel Ojjeh,Microsoft site server (commerce ed.): Talk-sli...,Microsoft Site Server (Commerce Edition),1999,1999,International Conference on Management of Data,SIGMOD Conference
3885,0.373410,Asuman Dogac,M. Tamer Özsu,Data management issues in electronic commerce,Data Management Issues in Electronic Commerce ...,2002,1999,ACM SIGMOD Record,SIGMOD Conference
9152,0.387697,Jim Melton,"Jim Melton, Andrew Eisenberg",Standards,Standards in Practice,2002,1998,ACM SIGMOD Record,SIGMOD Record
14520,0.207694,"Robert Meersman, Amit Sheth","Robert Meersman, Amit P. Sheth",Special section on semantic web and data manag...,Amicalola Report: Database and Information Sys...,2002,2002,ACM SIGMOD Record,SIGMOD Record
16975,0.302997,"Philip M. Lewis, Arthur Bernstein, Michael Kifer",Alfons Kemper,Databases and transaction processing: an appli...,Databases and Transaction Processing: An Appli...,2002,2002,ACM SIGMOD Record,SIGMOD Record
17025,0.320648,"Ian H. Witten, Eibe Frank",James Geller,Data mining: practical machine learning tools ...,Data Mining: Practical Machine Learning Tools ...,2002,2002,ACM SIGMOD Record,SIGMOD Record
17194,0.237184,Len Seligman,Leonard J. Seligman,Industry perspective,Industry Perspectives,2002,1997,ACM SIGMOD Record,SIGMOD Record
19182,0.376142,Aris Ouksel,Aris M. Ouksel,Mining the world wide web: an information sear...,Mining the World Wide Web: An Information Sear...,2002,2002,ACM SIGMOD Record,SIGMOD Record
20990,0.387697,Jim Melton,"Jim Melton, Andrew Eisenberg",Standards,Standards in Practice,2002,1998,ACM SIGMOD Record,SIGMOD Record


We see that even matches with lower best match scores were matched quite well. Seems that probabilistic record linkage approach works very effectively for these datasets. We'll drop matches below 0.3 best match score and check how many records we matched correctly:

In [ ]:
threshold = df_matched['best_match_score'] > 0.3
df_matched = df_matched[threshold]

In [ ]:
# Save matching results
df_matched[['__id_right', '__id_left']].to_csv('fuzzymatches.csv', index=False, header=['idDBLP', 'idACM'])

In [ ]:
# Read matches
df_fuzzymatches = pd.read_csv('fuzzymatches.csv')

## Record Linkage

### Full indexing

In [ ]:
# Re-read datasets using id columns as indices
df_ACM = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv', 
                     usecols=['id', 'title', 'authors', 'venue', 'year'], index_col='id')
df_DBLP = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', 
                     usecols=['id', 'title', 'authors', 'venue', 'year'], index_col='id')

In [ ]:
# Build the indexer
indexer = recordlinkage.Index()

# Try using full index
indexer.full()
candidates = indexer.index(df_ACM, df_DBLP)

In [ ]:
# Check the number of candidates
len(candidates)

6001104

In [ ]:
compare = recordlinkage.Compare()

compare.exact('year', 'year', label='year')
compare.string('authors', 'authors', label='authors',
               threshold=0.8, method='jarowinkler')
compare.string('title', 'title', label='title',
               threshold=0.8, method='jarowinkler')
compare.string('venue', 'venue', label='venue',
               threshold=0.8, method='jarowinkler')

compare_vectors = compare.compute(candidates, df_ACM, df_DBLP)

end = time.time()

time_elapsed_full = end - start

In [ ]:
# Seconds it took to run
time_elapsed_full

130.22705006599426

In [ ]:
compare_vectors.head()

year  authors  title  venue
id_1   id_2                                                 
304586 journals/sigmod/Mackay99     1      0.0    0.0    0.0
       conf/vldb/PoosalaI96         0      0.0    0.0    0.0
       conf/vldb/PalpanasSCP02      0      0.0    0.0    0.0
       conf/vldb/GardarinGT96       0      0.0    0.0    0.0
       conf/vldb/HoelS95            0      0.0    0.0    0.0

In [ ]:
# Score totals
compare_vectors.sum(axis=1).value_counts().sort_index()

0.0    5103659
1.0     857686
2.0      37881
3.0       1476
4.0        402
dtype: int64

In [ ]:
# Potential matches
potential_matches = compare_vectors[compare_vectors.sum(axis=1) > 1].reset_index()

In [ ]:
potential_matches['Score'] = potential_matches.loc[:, 'year':'venue'].sum(axis=1)
potential_matches.head()

,id_1,id_2,year,authors,title,venue,Score
0,304586,conf/sigmod/VossenW99,1,0.0,1.0,0.0,2.0
1,304587,conf/sigmod/CruzJ99,1,0.0,1.0,0.0,2.0
2,304589,conf/sigmod/BouguettayaBH99,1,0.0,1.0,0.0,2.0
3,304590,conf/sigmod/BaruGLMPVC99,1,0.0,1.0,0.0,2.0
4,304582,conf/sigmod/BrodskySCE99,1,1.0,1.0,0.0,3.0


In [ ]:
df_ACM.loc[304586]

title      The WASA2 object-oriented workflow management ...
authors                      Gottfried Vossen, Mathias Weske
venue         International Conference on Management of Data
year                                                    1999
Name: 304586, dtype: object

In [ ]:
df_DBLP.loc['conf/sigmod/VossenW99']

title      The WASA2 Object-Oriented Workflow Management ...
authors                      Mathias Weske, Gottfried Vossen
venue                                      SIGMOD Conference
year                                                    1999
Name: conf/sigmod/VossenW99, dtype: object

In [ ]:
# Add look up columns for easier comparison
df_ACM['ACM_lookup'] = df_ACM[['title',
                               'authors',
                               'venue',
                               'year']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

df_DBLP['DBLP_lookup'] = df_DBLP[['title',
                                  'authors',
                                  'venue',
                                  'year']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

In [ ]:
ACM_lookup = df_ACM['ACM_lookup'].reset_index()
ACM_lookup = ACM_lookup.rename({'id':'id_1'}, axis=1)

DBLP_lookup = df_DBLP['DBLP_lookup'].reset_index()
DBLP_lookup = DBLP_lookup.rename({'id':'id_2'}, axis=1)

In [ ]:
ACM_lookup.head()

,id_1,ACM_lookup
0,304586,The WASA2 object-oriented workflow management ...
1,304587,A user-centered interface for querying distrib...
2,304589,"World Wide Database-integrating the Web, CORBA..."
3,304590,XML-based information mediation with MIX_Chait...
4,304582,The CCUBE constraint object-oriented database ...


In [ ]:
DBLP_lookup.head()

,id_2,DBLP_lookup
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...


In [ ]:
ACM_merge = potential_matches.merge(ACM_lookup, how='left')

In [ ]:
ACM_merge.head()

,id_1,id_2,year,authors,title,venue,Score,ACM_lookup
0,304586,conf/sigmod/VossenW99,1,0.0,1.0,0.0,2.0,The WASA2 object-oriented workflow management ...
1,304587,conf/sigmod/CruzJ99,1,0.0,1.0,0.0,2.0,A user-centered interface for querying distrib...
2,304589,conf/sigmod/BouguettayaBH99,1,0.0,1.0,0.0,2.0,"World Wide Database-integrating the Web, CORBA..."
3,304590,conf/sigmod/BaruGLMPVC99,1,0.0,1.0,0.0,2.0,XML-based information mediation with MIX_Chait...
4,304582,conf/sigmod/BrodskySCE99,1,1.0,1.0,0.0,3.0,The CCUBE constraint object-oriented database ...


In [ ]:
final_merge = ACM_merge.merge(DBLP_lookup, how='left')

In [ ]:
cols = ['id_1', 'id_2', 'Score', 'ACM_lookup', 'DBLP_lookup']
final_merge[cols].sort_values(by=['id_1', 'Score'], ascending=False)

,id_1,id_2,Score,ACM_lookup,DBLP_lookup
35944,959087,journals/sigmod/Hainaut03,4.0,Research in database engineering at the Univer...,Research in database engineering at the Univer...
36001,959087,journals/sigmod/FifeG03,3.0,Research in database engineering at the Univer...,Research issues for data communication in mobi...
35923,959087,journals/sigmod/PitouraAPSV03,2.0,Research in database engineering at the Univer...,DBGlobe: a service-oriented P2P system for glo...
35924,959087,journals/sigmod/ChakravarthyAEFO03,2.0,Research in database engineering at the Univer...,Database Research at UT Arlington_Sharma Chakr...
35925,959087,journals/sigmod/GerlaX03,2.0,Research in database engineering at the Univer...,Multimedia streaming in large-scale sensor net...
...,...,...,...,...,...
23032,174639,journals/tods/WinslettSQ94,2.0,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Formal Query Languages for Secure Relational D...
23033,174639,journals/tods/YanG94,2.0,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Index Structures for Selective Dissemination o...
23034,174639,journals/tods/KorthS94,2.0,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Formal Aspects of Concurrency Control in Long-...
23035,174639,journals/tods/RosenthalR94,2.0,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Tools and Transformations - Rigorous and Other...


We see some matches, but the problem is that we have 39759 matched records, which is way more than we have records in both datasets, so for each id we'll leave only one match with the highest score.

In [ ]:
final_merge = final_merge[cols]

In [ ]:
indices = final_merge.groupby('id_1')['Score'].transform(max) == final_merge['Score']

In [ ]:
final_merge[indices]

,id_1,id_2,Score,ACM_lookup,DBLP_lookup
0,304586,conf/sigmod/VossenW99,2.0,The WASA2 object-oriented workflow management ...,The WASA2 Object-Oriented Workflow Management ...
1,304587,conf/sigmod/CruzJ99,2.0,A user-centered interface for querying distrib...,A User-Centered Interface for Querying Distrib...
2,304589,conf/sigmod/BouguettayaBH99,2.0,"World Wide Database-integrating the Web, CORBA...","World Wide Database - Integrating the Web, COR..."
3,304590,conf/sigmod/BaruGLMPVC99,2.0,XML-based information mediation with MIX_Chait...,XML-Based Information Mediation with MIX_Yanni...
4,304582,conf/sigmod/BrodskySCE99,3.0,The CCUBE constraint object-oriented database ...,The CCUBE Constraint Object-Oriented Database ...
...,...,...,...,...,...
39664,950482,conf/vldb/PottingerB03,2.0,"Guest editorial_Philip A. Bernstein, Yannis Io...",Merging Models Based on Given Correspondences_...
39665,950482,journals/sigmod/Aberer03c,2.0,"Guest editorial_Philip A. Bernstein, Yannis Io...",Guest editor's introduction_Karl Aberer_SIGMOD...
39666,672980,conf/vldb/Guting94,3.0,GraphDB: Modeling and Querying Graphs in Datab...,GraphDB: Modeling and Querying Graphs in Datab...
39688,945741,journals/sigmod/Anisimov03,4.0,Review of The data warehouse toolkit: the comp...,Review of The data warehouse toolkit: the comp...


In [ ]:
final_merge[indices][['id_1', 'id_2']].to_csv('recordlinkageFULLmatches.csv', 
                                              index=False, header=['idACM', 'idDBLP'])

### Block indexing

Now, let's try using different indexing for record linkage - blocking:

In [ ]:
# Re-read datasets using id columns as indices
df_ACM = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv', 
                     usecols=['id', 'title', 'authors', 'venue', 'year'], index_col='id')
df_DBLP = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', 
                     usecols=['id', 'title', 'authors', 'venue', 'year'], index_col='id')

In [ ]:
# Build the indexer
indexer = recordlinkage.Index()

# Try using block indexing
indexer.block(left_on='year', right_on='year')

candidates = indexer.index(df_ACM, df_DBLP)

In [ ]:
# Check the number of candidates
len(candidates)

601284

In [ ]:
# original
compare = recordlinkage.Compare()

compare.exact('year', 'year', label='year')
compare.string('authors', 'authors', label='authors',
               threshold=0.8, method='jarowinkler')
compare.string('title', 'title', label='title',
               threshold=0.8, method='jarowinkler')
compare.string('venue', 'venue', label='venue',
               threshold=0.8, method='jarowinkler')

start = time.time()
compare_vectors = compare.compute(candidates, df_ACM, df_DBLP)
end = time.time()

time_elapsed_full = end - start

In [ ]:
# parameter change
compare = recordlinkage.Compare()

compare.exact('year', 'year', label='year')
compare.string('authors', 'authors', label='authors',
               method='qgram')
compare.string('title', 'title', label='title',
               threshold=0.8, method='jarowinkler')
compare.string('venue', 'venue', label='venue',
               method='qgram')

start = time.time()
compare_vectors = compare.compute(candidates, df_ACM, df_DBLP)
end = time.time()

time_elapsed_full = end - start

In [ ]:
# Time spent
time_elapsed_full

82.66433072090149

In [ ]:
# Score totals
compare_vectors.sum(axis=1).value_counts().sort_index()

1.0    562778
2.0     36357
3.0      1685
4.0       464
dtype: int64

In [ ]:
# Potential matches
potential_matches = compare_vectors[compare_vectors.sum(axis=1) > 1].reset_index()

potential_matches['Score'] = potential_matches.loc[:, 'year':'venue'].sum(axis=1)
potential_matches.head()

,id_1,id_2,year,authors,title,venue,Score
0,304586,journals/sigmod/Mackay99,1,0.125000,0.0,0.042553,1.167553
1,304586,conf/sigmod/Keim99,1,0.062500,0.0,0.234043,1.296543
2,304586,conf/vldb/ChenCFGJMTW99,1,0.131148,0.0,0.000000,1.131148
3,304586,conf/sigmod/BraumandlKK99,1,0.137255,0.0,0.234043,1.371297
4,304586,conf/vldb/NinkHR99,1,0.105263,0.0,0.000000,1.105263


In [ ]:
# Add look up columns for easier comparison
df_ACM['ACM_lookup'] = df_ACM[['title',
                               'authors',
                               'venue',
                               'year']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

df_DBLP['DBLP_lookup'] = df_DBLP[['title',
                                  'authors',
                                  'venue',
                                  'year']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

In [ ]:
ACM_lookup = df_ACM['ACM_lookup'].reset_index()
ACM_lookup = ACM_lookup.rename({'id':'id_1'}, axis=1)

DBLP_lookup = df_DBLP['DBLP_lookup'].reset_index()
DBLP_lookup = DBLP_lookup.rename({'id':'id_2'}, axis=1)

In [ ]:
ACM_merge = potential_matches.merge(ACM_lookup, how='left')
final_merge = ACM_merge.merge(DBLP_lookup, how='left')

cols = ['id_1', 'id_2', 'Score', 'ACM_lookup', 'DBLP_lookup']
final_merge[cols].sort_values(by=['id_1', 'Score'], ascending=False)

,id_1,id_2,Score,ACM_lookup,DBLP_lookup
568046,959087,journals/sigmod/Hainaut03,3.777778,Research in database engineering at the Univer...,Research in database engineering at the Univer...
568249,959087,journals/sigmod/FifeG03,2.777778,Research in database engineering at the Univer...,Research issues for data communication in mobi...
568156,959087,journals/sigmod/Hammer03,1.954248,Research in database engineering at the Univer...,Treasurer's Message_Joachim Hammer_SIGMOD Reco...
568134,959087,journals/sigmod/Winslett03b,1.944444,Research in database engineering at the Univer...,Interview with Rakesh Agrawal_Marianne Winslet...
568218,959087,journals/sigmod/Winslett03a,1.944444,Research in database engineering at the Univer...,Interview with Michael Stonebraker_Marianne Wi...
...,...,...,...,...,...
479185,174639,journals/sigmod/X94,1.045455,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",The TSQL2 Final Language Definition Announceme...
479303,174639,journals/sigmod/X94a,1.045455,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Calls for Papers / Announcements_?_SIGMOD Reco...
479119,174639,conf/vldb/Bernstein94,1.032258,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",PC Database Systems - Present and Future_Phili...
479197,174639,conf/vldb/Lesk94,1.032258,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Experiments on Access to Digital Libraries: Ho...


We still have the similar amount of matches, so we will do the same as before - leave only matches with highest score:

In [ ]:
final_merge = final_merge[cols]
indices = final_merge.groupby('id_1')['Score'].transform(max) == final_merge['Score']
final_merge[indices]

,id_1,id_2,Score,ACM_lookup,DBLP_lookup
183,304586,conf/sigmod/VossenW99,3.171543,The WASA2 object-oriented workflow management ...,The WASA2 Object-Oriented Workflow Management ...
360,304587,conf/sigmod/CruzJ99,3.175219,A user-centered interface for querying distrib...,A User-Centered Interface for Querying Distrib...
504,304589,conf/sigmod/BouguettayaBH99,3.191489,"World Wide Database-integrating the Web, CORBA...","World Wide Database - Integrating the Web, COR..."
740,304590,conf/sigmod/BaruGLMPVC99,3.154043,XML-based information mediation with MIX_Chait...,XML-Based Information Mediation with MIX_Yanni...
1017,304582,conf/sigmod/BrodskySCE99,3.175219,The CCUBE constraint object-oriented database ...,The CCUBE Constraint Object-Oriented Database ...
...,...,...,...,...,...
589429,945735,journals/sigmod/Ouksel03,3.777778,In-context peer-to-peer information filtering ...,In-context peer-to-peer information filtering ...
589831,950484,journals/vldb/ChanGR03,2.984608,RE-tree: an efficient index structure for regu...,RE-tree: an efficient index structure for regu...
590157,950483,journals/vldb/RamamurthyDS03,3.078947,A case for fractured mirrors_Ravishankar Ramam...,"A case for fractured mirrors_David J. DeWitt, ..."
590282,950482,journals/vldb/BernsteinIR03,3.029767,"Guest editorial_Philip A. Bernstein, Yannis Io...","Guest editorial_Yannis E. Ioannidis, Raghu Ram..."


In [ ]:
final_merge[indices][['id_1', 'id_2']].to_csv('recordlinkageBLOCKmatchesUPDATED2.csv', 
                                              index=False, header=['idACM', 'idDBLP'])

### Sorted Neighbourhood indexing

And finally, let's try sorted neighbourhood indexing:

In [ ]:
# Re-read datasets using id columns as indices
df_ACM = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv', 
                     usecols=['id', 'title', 'authors', 'venue', 'year'], index_col='id')
df_DBLP = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', 
                     usecols=['id', 'title', 'authors', 'venue', 'year'], index_col='id')

In [ ]:
# Build the indexer
indexer = recordlinkage.Index()

# Try using sorted neighbourhood indexing
indexer.sortedneighbourhood(left_on='year', right_on='year')

candidates = indexer.index(df_ACM, df_DBLP)

In [ ]:
# Check the number of candidates
len(candidates)

1680430

In [ ]:
compare = recordlinkage.Compare()

compare.exact('year', 'year', label='year')
compare.string('authors', 'authors', label='authors',
               threshold=0.8, method='jarowinkler')
compare.string('title', 'title', label='title',
               threshold=0.8, method='jarowinkler')
compare.string('venue', 'venue', label='venue',
               threshold=0.8, method='jarowinkler')

start = time.time()
compare_vectors = compare.compute(candidates, df_ACM, df_DBLP)
end = time.time()

time_elapsed_full = end - start

In [ ]:
# Time spent
time_elapsed_full

36.27716588973999

In [ ]:
# Score totals
compare_vectors.sum(axis=1).value_counts().sort_index()

0.0    1017287
1.0     624129
2.0      37161
3.0       1451
4.0        402
dtype: int64

In [ ]:
# Potential matches
potential_matches = compare_vectors[compare_vectors.sum(axis=1) > 1].reset_index()

potential_matches['Score'] = potential_matches.loc[:, 'year':'venue'].sum(axis=1)
potential_matches.head()

,id_1,id_2,year,authors,title,venue,Score
0,310075,journals/sigmod/EisenbergM00,0,1.0,0.0,1.0,2.0
1,320249,journals/tods/FegarasM00,0,0.0,1.0,1.0,2.0
2,310062,journals/sigmod/TheodoridisN00,0,1.0,0.0,1.0,2.0
3,344864,journals/sigmod/EisenbergM00,0,1.0,0.0,1.0,2.0
4,306119,journals/sigmod/GatziuJSV99,0,0.0,1.0,1.0,2.0


In [ ]:
# Add look up columns for easier comparison
df_ACM['ACM_lookup'] = df_ACM[['title',
                               'authors',
                               'venue',
                               'year']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

df_DBLP['DBLP_lookup'] = df_DBLP[['title',
                                  'authors',
                                  'venue',
                                  'year']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

In [ ]:
ACM_lookup = df_ACM['ACM_lookup'].reset_index()
ACM_lookup = ACM_lookup.rename({'id':'id_1'}, axis=1)

DBLP_lookup = df_DBLP['DBLP_lookup'].reset_index()
DBLP_lookup = DBLP_lookup.rename({'id':'id_2'}, axis=1)

In [ ]:
ACM_merge = potential_matches.merge(ACM_lookup, how='left')
final_merge = ACM_merge.merge(DBLP_lookup, how='left')

cols = ['id_1', 'id_2', 'Score', 'ACM_lookup', 'DBLP_lookup']
final_merge[cols].sort_values(by=['id_1', 'Score'], ascending=False)

,id_1,id_2,Score,ACM_lookup,DBLP_lookup
35185,959087,journals/sigmod/Hainaut03,4.0,Research in database engineering at the Univer...,Research in database engineering at the Univer...
35241,959087,journals/sigmod/FifeG03,3.0,Research in database engineering at the Univer...,Research issues for data communication in mobi...
35165,959087,journals/sigmod/PitouraAPSV03,2.0,Research in database engineering at the Univer...,DBGlobe: a service-oriented P2P system for glo...
35166,959087,journals/sigmod/ChakravarthyAEFO03,2.0,Research in database engineering at the Univer...,Database Research at UT Arlington_Sharma Chakr...
35167,959087,journals/sigmod/GerlaX03,2.0,Research in database engineering at the Univer...,Multimedia streaming in large-scale sensor net...
...,...,...,...,...,...
28932,174639,journals/tods/WinslettSQ94,2.0,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Formal Query Languages for Secure Relational D...
28933,174639,journals/tods/YanG94,2.0,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Index Structures for Selective Dissemination o...
28934,174639,journals/tods/KorthS94,2.0,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Formal Aspects of Concurrency Control in Long-...
28935,174639,journals/tods/RosenthalR94,2.0,"Altruistic locking_Kenneth Salem, H&#233;ctor ...",Tools and Transformations - Rigorous and Other...


Once again, we'll leave matches with highest score:

In [ ]:
final_merge = final_merge[cols]
indices = final_merge.groupby('id_1')['Score'].transform(max) == final_merge['Score']
final_merge[indices]

,id_1,id_2,Score,ACM_lookup,DBLP_lookup
23,671327,conf/vldb/Johnson99,2.0,Performance Measurements of Tertiary Storage D...,Performance Measurements of Compressed Bitmap ...
55,375680,journals/tods/ChakrabartiKMP02,2.0,Locally adaptive dimensionality reduction for ...,Locally adaptive dimensionality reduction for ...
61,603881,journals/sigmod/RossFS02,2.0,Influential papers_Ken Ross_ACM SIGMOD Record ...,Reminiscences on Influential Papers_Kenneth A....
62,603881,journals/sigmod/RossAA02,2.0,Influential papers_Ken Ross_ACM SIGMOD Record ...,Reminiscences on Influential Papers_Kenneth A....
63,603882,journals/sigmod/RossFS02,3.0,Reminiscences on influential papers_Kenneth A....,Reminiscences on Influential Papers_Kenneth A....
...,...,...,...,...,...
38913,507368,journals/sigmod/LabrinidisM01,2.0,Career forum_Alexandros Labrinidis_ACM SIGMOD ...,Career-Enhancing Services at SIGMOD Online_Ale...
38914,507368,journals/sigmod/Labrinidis01,2.0,Career forum_Alexandros Labrinidis_ACM SIGMOD ...,Introduction to the Career Forum Column_Alexan...
38938,637432,journals/sigmod/MeltonE01,2.0,Standards_Jim Melton_ACM SIGMOD Record _2002,SQL Multimedia and Application Packages (SQL/M...
38939,637432,journals/sigmod/EisenbergM01,2.0,Standards_Jim Melton_ACM SIGMOD Record _2002,SQL/XML and the SQLX Informal Group of Compani...


In [ ]:
final_merge[indices][['id_1', 'id_2']].to_csv('recordlinkageSRTNGHBRmatches.csv', 
                                              index=False, header=['idACM', 'idDBLP'])

## Performance of different approaches

Let's compare performances of fuzzymatcher and recordlinkage with perfect matches.

In [ ]:
# perfect matches
perfect_mapping = pd.read_csv('perfectMapping.csv')

# fuzzymatcher
#fuzzy_matches = pd.read_csv('fuzzymatches.csv')

# recordlinkage
#rl_full_matches = pd.read_csv('recordlinkageFULLmatches.csv')
#rl_block_matches = pd.read_csv('recordlinkageBLOCKmatches.csv')
#rl_srtnghbr_matches = pd.read_csv('recordlinkageSRTNGHBRmatches.csv')

First, we'll check if matches contain duplicated ids.

In [ ]:
fuzzy_dps = len(fuzzy_matches) - len(fuzzy_matches['idDBLP'].unique())
rl_full_dps = len(rl_full_matches) - len(rl_full_matches['idDBLP'].unique())
rl_block_dps = len(rl_block_matches) - len(rl_block_matches['idDBLP'].unique())
rl_srtnghbr_dps = len(rl_srtnghbr_matches) - len(rl_srtnghbr_matches['idDBLP'].unique())

print('ID duplicates')
print(f'fuzzy: {fuzzy_dps}')
print(f'record linkage full: {rl_full_dps}')
print(f'record linkage block: {rl_block_dps}')
print(f'record linkage sorted neighbourhood: {rl_srtnghbr_dps}')

ID duplicates
fuzzy: 31
record linkage full: 2074
record linkage block: 2851
record linkage sorted neighbourhood: 2886


We can see that some ids in the DBLP dataset were matched with multiple ids in the ACM dataset. Matches that were made using record linkage have a lot id duplicates, which would distort our performance metrics, so we will remove the duplicates to have ids matched only once. Matched records achieved the highest matching scores, so we will not pay attention to which of the duplicates we will remove.

In [ ]:
fuzzy_matches = fuzzy_matches.drop_duplicates('idDBLP')
rl_full_matches = rl_full_matches.drop_duplicates('idDBLP')
rl_block_matches = rl_block_matches.drop_duplicates('idDBLP')
rl_srtnghbr_matches = rl_srtnghbr_matches.drop_duplicates('idDBLP')

In [ ]:
fuzzy_correct_matches = fuzzy_matches.merge(perfect_mapping, on=['idDBLP', 'idACM'], how='inner')
rl_full_correct_matches = rl_full_matches.merge(perfect_mapping, on=['idDBLP', 'idACM'], how='inner')
rl_block_correct_matches = rl_block_matches.merge(perfect_mapping, on=['idDBLP', 'idACM'], how='inner')
rl_srtnghbr_correct_matches = rl_srtnghbr_matches.merge(perfect_mapping, on=['idDBLP', 'idACM'], how='inner')

In [ ]:
precision = len(fuzzy_correct_matches) / len(fuzzy_matches)
recall = len(fuzzy_correct_matches) / len(perfect_mapping)
f1 = 2 * (precision * recall) / (precision + recall)

print("Fuzzy Matcher performance:")
print(f'precision: {precision}')
print(f'recall: {recall}')
print(f'f1: {f1}')

Fuzzy Matcher performance:
precision: 0.9927239654388358
recall: 0.9815647482014388
f1: 0.9871128193533801


In [ ]:
precision = len(rl_full_correct_matches) / len(rl_full_matches)
recall = len(rl_full_correct_matches) / len(perfect_mapping)
f1 = 2 * (precision * recall) / (precision + recall)

print("Record Linkage full indexing performance:")
print(f'precision: {precision}')
print(f'recall: {recall}')
print(f'f1: {f1}')

Record Linkage full indexing performance:
precision: 0.8101374570446735
recall: 0.8480215827338129
f1: 0.828646748681898


In [ ]:
precision = len(rl_block_correct_matches) / len(rl_block_matches)
recall = len(rl_block_correct_matches) / len(perfect_mapping)
f1 = 2 * (precision * recall) / (precision + recall)

print("Record Linkage block indexing performance:")
print(f'precision: {precision}')
print(f'recall: {recall}')
print(f'f1: {f1}')

Record Linkage block indexing performance:
precision: 0.8114507102884202
recall: 0.8475719424460432
f1: 0.8291180998460523


In [ ]:
precision = len(rl_srtnghbr_correct_matches) / len(rl_srtnghbr_matches)
recall = len(rl_srtnghbr_correct_matches) / len(perfect_mapping)
f1 = 2 * (precision * recall) / (precision + recall)

print("Record Linkage sorted neighbourhood indexing performance:")
print(f'precision: {precision}')
print(f'recall: {recall}')
print(f'f1: {f1}')

Record Linkage sorted neighbourhood indexing performance:
precision: 0.8082545141874462
recall: 0.8453237410071942
f1: 0.8263736263736263


We can see that we have achieved very good scores using both Record Linkage and Fuzzy Matcher packages. However, there is still a noticeable difference - Fuzzy Matcher achieved an outstanding precision of around 0.99, 0.98 recall and 0.99 f1 score, which is around 0.18, 0.14 and 0.16 higher, respectively, than records matched using Record Linkage methods. <br><br> Between three indexing methods of Record Linkage matching, block indexing achieved the highest precision (i.e. 0.811) and f1 score (i.e. 0.829) while full indexing achieved the highest recall (i.e. 0.848), although the differences are comparatively small. Even though the performance differences between indexing methods are insignificant, time consumption varies substantially - full indexing took 130 seconds to complete, block indexing took 13 seconds and sorted neighbourhood took 36 seconds. Taken into account time consumption and performance, we can conclude that for this specific data linkage problem block indexing is the best option out of three indexing methods. <br><br> Let's go back and try changing the parameters of block indexing method and check the results.

With parameters: <br>
authors: jaro, title: levensthein, venue: jaro-wrinkler

In [ ]:
rl_block_updated_matches = pd.read_csv('recordlinkageBLOCKmatchesUPDATED.csv')

In [ ]:
rl_block_updated_matches = rl_block_updated_matches.drop_duplicates('idDBLP')
rl_block_updated_correct_matches = rl_block_updated_matches.merge(perfect_mapping, on=['idDBLP', 'idACM'], how='inner')

In [ ]:
precision = len(rl_block_updated_correct_matches) / len(rl_block_updated_matches)
recall = len(rl_block_updated_correct_matches) / len(perfect_mapping)
f1 = 2 * (precision * recall) / (precision + recall)

print("Record Linkage block indexing with different parameters performance:")
print(f'precision: {precision}')
print(f'recall: {recall}')
print(f'f1: {f1}')

Record Linkage block indexing with different parameters performance:
precision: 0.8206778206778207
recall: 0.8601618705035972
f1: 0.8399560922063667


Let's check the results with different parameters again. (authors: q-gram, title: jarowrinkler, venue: q-gram)

In [ ]:
rl_block_new = pd.read_csv('recordlinkageBLOCKmatchesUPDATED2.csv')

In [ ]:
rl_block_new = rl_block_new.drop_duplicates('idDBLP')
rl_block_new_correct = rl_block_new.merge(perfect_mapping, on=['idDBLP', 'idACM'], how='inner')

In [ ]:
precision = len(rl_block_new_correct) / len(rl_block_new)
recall = len(rl_block_new_correct) / len(perfect_mapping)
f1 = 2 * (precision * recall) / (precision + recall)

print("Record Linkage block indexing with different parameters performance:")
print(f'precision: {precision}')
print(f'recall: {recall}')
print(f'f1: {f1}')

Record Linkage block indexing with different parameters performance:
precision: 0.9390728476821192
recall: 0.9563848920863309
f1: 0.9476498106482514


Nice! With couple settings changed we managed to achieve even better performance. However, it took 82 seconds to complete the matching, which is 69 seconds longer than with previous parameters, but considering an increase in all three performance metrics, we can say that this one is the most suitable option. <br> <br> We could probably make further improvements by exploring similarity or distance measures and thresholds and we might even reach or at least close the gap to the performance of Fuzzy Matcher, but for now our rankings are:


1.   Fuzzy Matcher
2.   Record Linkage with block indexing
3.   Record Linkage with full / sorted neighbourhood indexing

Keep in mind that block indexing worked well for this specific problem, because publication year had to match exactly for the records to be matched, thus reducing search space to the same publication year automatically did the first important step towards correct matches.

